In [11]:
import random

import networkx as nx
import numpy as np
from utils_cv import compute_n, is_cycle
from utils import get_distance_table, get_distance

In [12]:
def get_permutation(random_key_sequence):
    key = np.array(random_key_sequence)
    return np.argsort(key)[::-1]

In [13]:
def decode(permutation, degree_constrained):
    n = compute_n(len(permutation))
    G = nx.complete_graph(n)
    # print("Full G:", G.edges)
    # print(G.edges[4])
    edges = list(G.edges)

    T = nx.empty_graph(n)
    # print(T.nodes)
    i = 0
    while len(T.edges) < n - 1:
        # T.add_edge(G.edges[i])
        j = permutation[i]
        # print(j)
        i += 1
        edge = edges[j]
        # print(edge)
        T.add_edge(*edge)
        nodes = [*edge]
        # print("Edge:", edge)
        # print("T:", T.edges)
        # print("Circle:", is_cycle(T))
        # print("Constrain:", T.degree(nodes[0]) > degree_constrained, T.degree(nodes[1]) > degree_constrained)
        if is_cycle(T):
            T.remove_edge(*edge)
        elif T.degree(nodes[0]) > degree_constrained or T.degree(nodes[1]) > degree_constrained:
            T.remove_edge(*edge)

    return T

In [14]:
def gen_key(n):
    l = int(n * (n - 1) / 2)
    return [random.random() for _ in range(l)]

In [15]:
def calculate_fitness(random_key_sequence, degree_constrained, distances_table):
    permutation = get_permutation(random_key_sequence)
    tree = decode(permutation, degree_constrained)
    edges = tree.edges
    # # Check if the degrees more than target degrees
    # if any(x > degree_constrained for x in degrees):
    #     return 9999999

    cost = 0
    for edge in edges:
        cost = cost + get_distance(edge, distances_table)
    return cost

In [16]:
def crossover(parent1, parent2, crossover_rate=0.8):
    offspring1 = parent1[:]
    offspring2 = parent2[:]
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 2)
        offspring1[crossover_point:], offspring2[crossover_point:] = offspring2[crossover_point:], offspring1[
                                                                                                   crossover_point:]
    return offspring1, offspring2


In [17]:


def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual


def mutate_pop(population, mutation_rate):
    new_pop = []
    for individual in population:
        if random.random() < mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
            new_pop.append(individual)
    return new_pop

In [18]:
def get_new_pop(population, population_size, degree_constrained, distances_table):
    fitness_values = [calculate_fitness(individual, degree_constrained, distances_table) for
                      individual in population]
    sorted_fitness = sorted(fitness_values)
    index_dict = {val: idx for idx, val in enumerate(fitness_values)}
    # Lấy danh sách index của population đã được sắp xếp theo fitness
    pop_index = [index_dict[val] for val in sorted_fitness]
    # Lấy index của [population_size] phần tử đầu (có cost nhỏ nhất)
    new_pop_index = pop_index[:population_size]
    new_pop = [population[i] for i in new_pop_index]
    return new_pop

In [19]:

def run_ga(n, degree_constrained, distances_table, population_size=50, crossover_rate=0.8, mutation_rate=0.1,
           max_generations=50):
    population = [gen_key(n) for _ in range(population_size)]
    # print(population)
    for generation in range(max_generations):
        # print(population)
        # Tính một mảng fitness value của population
        fitness_values = [calculate_fitness(cv_sequence, degree_constrained, distances_table) for
                          cv_sequence in population]
        # print(fitness_values)
        new_population = []
        while len(new_population) < population_size:
            i, j = np.random.choice(range(population_size), size=2, replace=False,
                                    p=np.array(fitness_values) / sum(fitness_values))
            parent1 = population[i]
            parent2 = population[j]
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        for i in range(len(new_population)):
            mutate(new_population[i])
        population = population + new_population
        population = get_new_pop(population, population_size, degree_constrained, distances_table)

    return population


In [20]:
degree_constrained = 2
n = 5
distances_table = get_distance_table("data/7_wi29.csv")

population = run_ga(n, degree_constrained, distances_table)
print(population[0])
print(calculate_fitness(population[0], degree_constrained, distances_table))

[0.8321914066231066, 0.08565159367768627, 0.22778066237543593, 0.31395578293752546, 0.014521902128260877, 0.6566869533920249, 0.965158728367455, 0.40749676203550755, 0.6373198008698537, 0.9288953678518274]
4277.163670389418


In [21]:
T = decode(get_permutation(population[0]), degree_constrained)
print(T.edges)
print(nx.is_tree(T))


[(0, 1), (1, 4), (2, 3), (3, 4)]
True
